<a href="https://colab.research.google.com/github/zhoudanxie/regulatory_data_repository/blob/main/Unified_Agenda_Data_Automation_demo2_v3_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load Library

In [ ]:
# library
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os
import numpy as np
import xml.etree.cElementTree as et
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 10)
from lxml import etree
import requests
import ipywidgets as widgets

## Data Automation

In [ ]:
def replace_noun(text):
    if text==None:
        text='N/A'
    else:
        text=text
    return text

# %% Revised function
def xml_to_csv(file):

    # Create empty lists to store values
    agenda_date,RIN,agency_code,agency_name,department_code,department_name,rule_title,abstract,\
        priority,RIN_status,rule_stage,major,CFR,legal_authority,legal_deadline_list,action_list= ([] for i in range(16))

    # Parse XML
    parser = etree.XMLParser(encoding="UTF-8", recover=True)
    parsed_xml = etree.parse(file, parser)  # prevent form issue
    root = parsed_xml.getroot()

    for child in root:
        agenda_date.append(child.find('PUBLICATION')[0].text)
        RIN.append(child.find('RIN').text)
        agency_code.append(child.find('AGENCY')[0].text)

        if child.find('AGENCY').find('NAME') != None:
            agency_name.append(child.find('AGENCY').find('NAME').text)
        else:
            agency_name.append('')

        if child.find('PARENT_AGENCY') != None:
            department_code.append(child.find('PARENT_AGENCY')[0].text)
            department_name.append(child.find('PARENT_AGENCY')[1].text)
        else:
            department_code.append('')
            department_name.append('')

        rule_title.append(child.find('RULE_TITLE').text)
        abstract.append(child.find('ABSTRACT').text)

        if child.find('PRIORITY_CATEGORY') != None:
            priority.append(child.find('PRIORITY_CATEGORY').text)
        else:
            priority.append('')
        if child.find('RIN_STATUS') != None:
            RIN_status.append(child.find('RIN_STATUS').text)
        else:
            RIN_status.append('')
        if child.find('RULE_STAGE') != None:
            rule_stage.append(child.find('RULE_STAGE').text)
        else:
            rule_stage.append('')
        if child.find('MAJOR') != None:
            major.append(child.find('MAJOR').text)
        else:
            major.append('')

        if child.find('CFR_LIST') != None:
            index = 0
            cfr_text = ''
            while (index < len(list(child.find('CFR_LIST')))):
                add = child.find('CFR_LIST')[index].text
                if cfr_text == '':
                    cfr_text = add
                else:
                    cfr_text = cfr_text + "; " + str(add)
                index = index + 1
            CFR.append(cfr_text)
        else:
            CFR.append('')

        if child.find('LEGAL_AUTHORITY_LIST') != None:
            index = 0
            lauth_text = ''
            while (index < len(list(child.find('LEGAL_AUTHORITY_LIST')))):
                add = child.find('LEGAL_AUTHORITY_LIST')[index].text
                if lauth_text == '':
                    lauth_text = add
                else:
                    lauth_text = lauth_text + "; " + str(add)
                index = index + 1
            legal_authority.append(lauth_text)
        else:
            legal_authority.append('')

        if child.find('LEGAL_DLINE_LIST') is not None:
            legal_deadlines = []
            if child.find('LEGAL_DLINE_LIST').find('LEGAL_DLINE_INFO') != None:
                for element in child.find('LEGAL_DLINE_LIST').findall('LEGAL_DLINE_INFO'):
                    lddl_text = replace_noun(element.find('DLINE_TYPE').text) + '; ' + \
                                replace_noun(element.find('DLINE_ACTION_STAGE').text) + '; ' + \
                                replace_noun(element.find('DLINE_DATE').text) + '; ' + \
                                replace_noun(element.find('DLINE_DESC').text)
                    legal_deadlines.append(lddl_text)
            legal_deadline_list.append(legal_deadlines)
        else:
            legal_deadline_list.append([])

        if child.find('TIMETABLE_LIST') != None:
            actions=[]
            for element in child.find('TIMETABLE_LIST').findall('TIMETABLE'):
                if element.find('FR_CITATION') != None:
                    action_text = element.find('TTBL_ACTION').text + '; ' + \
                                    element.find('TTBL_DATE').text + '; ' + \
                                    element.find('FR_CITATION').text
                else:
                    if element.find('TTBL_DATE') != None:
                        action_text = element.find('TTBL_ACTION').text + '; ' + \
                                        element.find('TTBL_DATE').text
                    else:
                        action_text = element.find('TTBL_ACTION').text
                actions.append(action_text)
            action_list.append(actions)
        else:
            action_list.append([])

    # Convert lists to a dataframe
    df_xml=pd.DataFrame(list(zip(agenda_date,RIN,agency_code,agency_name,department_code,department_name,\
                        rule_title,abstract,priority,RIN_status,rule_stage,major,CFR,legal_authority,\
                        legal_deadline_list,action_list)),\
              columns=['agenda_date','RIN','agency_code','agency_name','department_code','department_name',\
                        'rule_title','abstract','priority','RIN_status','rule_stage','major','CFR','legal_authority',\
                        'legal_deadline_list','action_list'])

    # Split legal deadline and action columns
    lddl_max = max([len(l) for l in df_xml['legal_deadline_list']])
    lddl_cols = []
    for i in range(1, lddl_max + 1):
        lddl_cols.append('legal_deadline' + str(i))
    df_xml[lddl_cols] = pd.DataFrame(df_xml['legal_deadline_list'].tolist(), index=df_xml.index)

    action_max = max([len(l) for l in df_xml['action_list']])
    action_cols = []
    for i in range(1, action_max + 1):
        action_cols.append('action' + str(i))
    df_xml[action_cols] = pd.DataFrame(df_xml['action_list'].tolist(), index=df_xml.index)

    df_xml.drop(['legal_deadline_list','action_list'],axis=1,inplace=True)

    return df_xml



#%% Define a function to download XML and convert to CSV within a given time interval (based on user input)
def time_interval_transform(start_year,start_season,end_year,end_season):



    files_failed=[]
    result_xml = []
    result_csv = []
    single_xml = []
    single_csv = []
    sea_option = ['04','10']

    # season str to int conversion
    def season_transform(s):
      season = ['04', '10']
      if s == 'Fall':
            return season[1]   #this is an str

      elif s == 'Spring':
          return season[0]
         #this is a str
      else:
        return f'Wrong season value, please only enter "Spring" and "Fall" for season option'

    season1 = season_transform(start_season)
    season2 = season_transform(end_season)

    ### single download
    def auto_single_download(year,season):

      season = season_transform(season)
      file_path = '/content/' + f'REGINFO_RIN_DATA_{year}{season}.xml'
      files_failed=[]

      try:
        if not os.path.exists(file_path):
          file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_{year}{season}.xml'
          r = requests.get(file_url, allow_redirects=True)
          open(file_path, 'wb').write(r.content)
          df = xml_to_csv(file_path) # transfer xml file to dataframe
          return df
          print(f'{year}{season} has been downloaded')

        else:
          print( f'File already exists in the directory')
          df = xml_to_csv(file_path) # transfer existed xml file to dataframe
          return df

      except:
        files_failed.append(f'{year}{season}')
        print(f'{year}{season} cannot be downloaded')

    #return df



    #condition 1: Extract single/both season file from same year
    if (end_year == start_year):

      if start_year == 2012:

            try:
              file_path1 = '/content/' + f'REGINFO_RIN_DATA_2012.xml'
              if not os.path.exists(file_path1):
                file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_2012.xml'
                r = requests.get(file_url, allow_redirects=True)
                open(file_path1, 'wb').write(r.content)

                df = xml_to_csv(file_path1)
                df.to_csv(f'/content/REGINFO_RIN_DATA_2012.csv')

                print(f'2012 has been downloaded')

              else:
                df = xml_to_csv(file_path1)
                df.to_csv(f'/content/REGINFO_RIN_DATA_2012.csv')
                print( f'2012 already exists in the directory')

            except:
                files_failed.append(f'2012')
                print(f'2012 cannot be downloaded')
                pass



      else:

        if (start_season==end_season):

          single_file = auto_single_download(start_year, start_season)
          single_file.to_csv(f'/content/REGINFO_RIN_DATA_{start_year}{start_season}.csv')
          print(f'{start_year} {start_season} has been downloaded')

        else:

          sea_1 = auto_single_download(start_year, start_season)
          sea_2 = auto_single_download(end_year, end_season)

          df_single_res = pd.concat([sea_1,sea_2], ignore_index=True)

          df_single_res.to_csv(f'/content/REGINFO_RIN_DATA_{start_year}{start_season}&{end_season}.csv', index=False)
          print(f'A CSV file for Unified Agenda {start_year}{start_season}&{end_season}.csv has been created!')








    # condition 2: for start year
    # 2012 check
    elif (start_year != end_year) : # to indicate specific condition
      if start_year == 2012:

            try:
              file_path1 = '/content/' + f'REGINFO_RIN_DATA_2012.xml'
              if not os.path.exists(file_path1):
                file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_2012.xml'
                r = requests.get(file_url, allow_redirects=True)
                open(file_path1, 'wb').write(r.content)
                result_xml.append(file_path1)

                print(f'2012 has been downloaded')

              else:
                print( f'2012 already exists in the directory')
                result_xml.append(file_path1) # appended file if already existed

            except:
                files_failed.append(f'2012')
                print(f'2012 cannot be downloaded')
                pass

          # years other than 2012
      else:

        if season1 == '10':


              file_path = '/content/' + f'REGINFO_RIN_DATA_{start_year}{season1}.xml'

              try:
                if not os.path.exists(file_path):
                  file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_{start_year}{season1}.xml'
                  r = requests.get(file_url, allow_redirects=True)
                  open(file_path, 'wb').write(r.content)
                  result_xml.append(file_path)

                  print(f'{start_year}{season1} has been downloaded')

                else:
                  print( f'{start_year}{season1} already exists in the directory')
                  result_xml.append(file_path) # appended exist file

              except:
                  files_failed.append(f'{start_year}{season1}')
                  print(f'{start_year}{season1} cannot be downloaded')
                  pass

        else:

          for i in sea_option:
            file_path = '/content/' + f'REGINFO_RIN_DATA_{start_year}{i}.xml'


            try:
              if not os.path.exists(file_path):
                file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_{start_year}{i}.xml'
                r = requests.get(file_url, allow_redirects=True)
                open(file_path, 'wb').write(r.content)
                result_xml.append(file_path)

                print(f'{start_year}{i} has been downloaded')

              else:
                print( f'{start_year}{i} already exists in the directory')
                result_xml.append(file_path) # appended exist file

            except:
                files_failed.append(f'{start_year}{i}')
                print(f'{start_year}{i} cannot be downloaded')
                pass




    # Condition 3: For the end year

    # 2012 check
      if end_year == 2012:

            try:
              file_path1 = '/content/' + f'REGINFO_RIN_DATA_2012.xml'
              if not os.path.exists(file_path1):
                file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_2012.xml'
                r = requests.get(file_url, allow_redirects=True)
                open(file_path1, 'wb').write(r.content)
                result_xml.append(file_path1)

                print(f'2012 has been downloaded')

              else:
                print( f'2012 already exists in the directory')
                result_xml.append(file_path) # appended exist file

            except:
                files_failed.append(f'2012')
                print(f'2012 cannot be downloaded')
                pass

      # years other than 2012
      else:

        if season2 == '04':

              file_path = '/content/' + f'REGINFO_RIN_DATA_{end_year}{season2}.xml'

              try:
                if not os.path.exists(file_path):
                  file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_{end_year}{season2}.xml'
                  r = requests.get(file_url, allow_redirects=True)
                  open(file_path, 'wb').write(r.content)
                  result_xml.append(file_path)

                  print(f'{end_year}{season2} has been downloaded')

                else:
                  print( f'{end_year}{season2} already exists in the directory')
                  result_xml.append(file_path) # appended exist file

              except:
                  files_failed.append(f'{end_year}{season2}')
                  print(f'{end_year}{season2} cannot be downloaded')
                  pass

        else:

          for i in sea_option:
            file_path = '/content/' + f'REGINFO_RIN_DATA_{end_year}{i}.xml'


            try:
              if not os.path.exists(file_path):
                file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_{end_year}{i}.xml'
                r = requests.get(file_url, allow_redirects=True)
                open(file_path, 'wb').write(r.content)
                result_xml.append(file_path)

                print(f'{end_year}{i} has been downloaded')

              else:
                print( f'{end_year}{i} already exists in the directory')
                result_xml.append(file_path) # appended exist file

            except:
                files_failed.append(f'{end_year}{i}')
                print(f'{end_year}{i} cannot be downloaded')
                pass



    # Condition 4: For all the years between the start and end years



      for year in range((start_year+1), end_year):

          if (end_year - start_year == 1): # skip this for loop if this condition is satisfied
            break

          # 2012 check
          if year == 2012:

            try:
              file_path1 = '/content/' + f'REGINFO_RIN_DATA_2012.xml'
              if not os.path.exists(file_path1):
                file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_2012.xml'
                r = requests.get(file_url, allow_redirects=True)
                open(file_path1, 'wb').write(r.content)
                result_xml.append(file_path1)

                print(f'2012 has been downloaded')

              else:
                print( f'2012 already exists in the directory')
                result_xml.append(file_path) # appended exist file

            except:
                files_failed.append(f'2012')
                print(f'2012 cannot be downloaded')
                pass

          # years other than 2012
          else:
            for i in sea_option:
              file_path = '/content/' + f'REGINFO_RIN_DATA_{year}{i}.xml'
              try:
                if not os.path.exists(file_path):
                  file_url = f'https://www.reginfo.gov/public/do/XMLViewFileAction?f=REGINFO_RIN_DATA_{year}{i}.xml'
                  r = requests.get(file_url, allow_redirects=True)
                  open(file_path, 'wb').write(r.content)
                  result_xml.append(file_path)
                  print(f'{year}{i} has been downloaded')
                else:
                  print( f'{year}{i} already exists in the directory')
                  result_xml.append(file_path) # appended exist file

              except:
                files_failed.append(f'{year}{i}')
                print('END!','Years failed:',files_failed)
                pass

    # Convert all downloaded XML files into a single CSV file

      for j in result_xml:
       new_csv = xml_to_csv(j)
       result_csv.append(new_csv)

      df_res = pd.concat(result_csv, ignore_index=True)
      df_res.to_csv(f'/content/REGINFO_RIN_DATA_{start_year}-{end_year}.csv', index=False)

      print(f'A CSV file for Unified Agenda {start_year} {start_season} - {end_year} {end_season} has been created!')

    return



In [ ]:
# update to most recent time info
import requests
from bs4 import BeautifulSoup
import re



# Make a request
page = requests.get(
    "https://www.reginfo.gov/public/do/eAgendaXmlReport")
soup = BeautifulSoup(page.content, 'html.parser')

# Extract newest file time information
newest_file_info = soup.select('li')[0].text[1:-6]

#extract newest year and season
current_year_season = re.split("\s", newest_file_info, 1) #list
current_year = int(current_year_season[1]) # int
current_season = current_year_season[0] # str
print(current_year_season,
current_year_season[0], # current season
current_year_season[1])  # current year

## User Input

In [ ]:

print(f'Please enter the year and season range from 1995 Fall to latest {current_year} {current_season}')
start_year = int(input('Please enter the start year of your search: '))
start_season = input('Please enter the season of start year ("Spring" or "Fall"): ')
end_year = int(input('Please enter the end year of your search: '))
end_season = input('Please enter the season of end year ("Spring" or "Fall"): ')



range_1 = range(1995,current_year+1)

if (start_year not in range_1) or (end_year not in range_1) :
  print(f'{start_year} and {end_year} is not in time range. \nPlease re-enter the year and season range between 1995 Fall and {current_year} {current_season}')

else:

  if start_year == 1995 and start_season != 'Fall':
    print('The file of year 1995 only have season Fall, please only enter Fall for year 1995 file')
  elif end_year == current_year:
     if end_season == current_season == 'Fall':
      time_interval_transform(start_year,start_season,end_year,end_season)
     elif current_season == 'Spring':
      if end_season != current_season:
        print(f'The most latest file time is {current_year} Spring on the websit. \nIf extracting latest file, please only use Spring for latest file season  ')
      else:
        time_interval_transform(start_year,start_season,end_year,end_season)
     else:
      print('Please only enter Spring and Fall for season option')
  else:
    time_interval_transform(start_year,start_season,end_year,end_season)





To-Do List, 7/10/2023:
- when inputing 2012 Spring and 2012 Fall, it does not show the message "A CSV file for ... has been created."
- use lower() for start_season and end_season to make it case insensitive
- Revise message output to be error-free and user-friendly
- Refine the code to be more efficient
- Put all the code into one single cell (so the user only needs to hit Run once)
- Explore ways other than Google Colab to allow users to run code
- Develop similar code for OIRA review data